# Seminar on Software Developmenet Tools
The first sections will be about quality, and the last one will deal with design.

## 0. Coding Environments
Running python from the terminal, running ipython, running jupyter notebook. There's a place for interactive computing. Do not marry your tools: don't use a hammer with a screw.

## 1. Debbuging
Go through the examples on `./debugging`; follow the instructions on the `README.md` if needed.

## 2. Virtual Environment
Show `virtualenv`, `virtualenvwrapper` and difference in numpy versions with this repo and `complex_opinion`.

Use:
```bash
pip show numpy | grep ^Version
```

**Note:** The character `^` in the command is a *regular expression anchor* that matches the *start of a line*, i.e., in this case `grep ^Version` matches any line that starts with the word `Version`.

The idea is to show that we can have different versions of `numpy` coexisting in our computer thanks to having multiple virtual environments.

## 2. Version Control
Show Git with `complex_opinion`. Delete some piece of code and show `status`, `diff`, and `restore`. Mention `log`, `pull`, and `branch`, and the fact that you can always go back to any point in history. Mention it's joint use with LaTeX for writing a paper, in particular the use of branches and `diff` for seeing changes and merging if agreed.

## 3. GitHub
A Hub for Git repositories. That's how we can get other people's code and colaborate between us (think of papers). Show PyPI too.

## 4. Packing
Useful for having your code available as third-party packages (explain editable installation: `pip install --editable .`), publishing in PyPI, and for **portability**, e.g., running in a cluster. In this last case, we just install micromamba, create the invironment, and do `pip install paquete.whl`. Show `dist` directory in `tumorsphere_culture`.

For packing:
1. Install everything necessary `pip install --upgrade build`
2. Build with `python3 -m build`

This will generate the files for distribution: `.tar.gz` for source distribution and `.whl` for built distribution.

Explain:
- `MANIFEST.in`
- `pyproject.toml`, in particular the part where we declare the dependencies

Note: if you're not distributing (and therefore maintaining) the software, it always a good idea to do a `pip freeze > requirements.txt` to be able to go back to the original package versions.

## 5. Testing
It's useless as quality assurance that “you wrote the code yourself and know how it works”. How do **I** know that it works? Tests are things that we check about the behavior of our code. By seeing which tests pass, I can get some level of assurance on the correctness of the code.

### 5.1 Unit Tests
Show example in the `testing` directory: `pytest -v test_math_utils.py`. This are examples of “unit tests”, tests that check a single instance of a property.

### 5.2 Integration Tests
Integration tests check the fuctioning of the system as a hole, not just of a single part. E.g., I could have checked that both my `Cell` and `Culture` objects are instanced correctly, but have my simulation do something wrong with them. See [meme](https://images.app.goo.gl/vH7UYtYb4a9cDsrM8).

An example of an integration test could be to reproduce a previous result which we trust. Show tests in `complex_opinion`. We check single properties of errors that should be rise when inappropriately instantiating objects (unit tests), but then we check that Mahdi's results are reproduced (integration tests). Use: `pytest -v tests/ --cov complex_opinion/ -m "not slow"`.

Note that the deselected tests are parametrized (explain that).

### 5.3 Coverage
In general, one should test extensively. We can measure the percentage of lines (and logical branches) that are tested by our code. Show `margin`.

### 5.4 Property-Based Testing
There are many more advanced types of testing. I'm just gonna show PBT here, which consists on trying to enforce a general property which is automatically challenged by dynamically generated unit tests. This test cases are generated with the `strategy` object provided by `hypothesis`, and try to find limit and pathological cases.

E.g., we can test symmetry for a given distance (try to avoid looking at the distance function, it's none of your business).

What's interesting about `hypothesis` is that not only tries test cases we wouldn't try, but it also can reduce counterexamples to a minimal case that's returned to the user: show summation function example.

### 5.5 Mutation Testing
Do not show (it's too slow), just comment.

Note that coverage percentage and mutant survival ratio are measures of the quality of *our tests*.

In [ ]:
# Look at how funny these attempts are
from hypothesis.strategies import text; print(text().example())

³þo


## 6. Style
The more we standardize, the less we have to think. If you get paid to think, that's just free money.

> You can have your car any color you want, as long as it's black.

Comply with [PEP-8, the Style Guide for Python Code](https://peps.python.org/pep-0008/). Show [Black]() and [numpy style for docstrings](https://numpydoc.readthedocs.io/en/latest/format.html).

## 7. Tox (Orchestration)
The solution to “it works on my machine...”

Run `tox` in `complex_opinion`. Show how a package can work with some Python versions and not with others (e.g., due to attempting to use the warlus with an eralier version than 3.8).

## 8. Software Design
Overview:
- The Zen of Python
- Basics of OOP
- The pilars of OOP
- Features of Good Design
- Design Principles
- SOLID
- Others (DRY, KISS, YAGNI)
- Classification of Design Patterns
- Selected Patterns

### 8.1 The Zen of Python

In [ ]:
import this

### 8.2 Basics of OOP
#### 8.2.1 What is an Object?
Objects are data structures that have:
1. Identity (i.e., a unique corresponding place in memory)
2. State (i.e., they can store data, called “attributes” in this context)
3. Behavior (i.e., they can have functions called “methods” in this context)

Let's create a basic object in python and test this properties.

In [ ]:
class BankAccount:
    def __init__(self):
        self.balance = 0  # Instance attribute: private to this object

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

    def get_balance(self):
        return self.balance

In [ ]:
account_of_pepe = BankAccount()

In [ ]:
# 1. Identity
id(account_of_pepe)

In [ ]:
# 2. State
print(account_of_pepe.balance)

In [ ]:
# 3. Behavior
print(f'Current balance: {account_of_pepe.balance}')
account_of_pepe.deposit(100)
print(f'Balance after deposit: {account_of_pepe.balance}')
account_of_pepe.withdraw(40)
print(f'Balance after withdrawal: {account_of_pepe.balance}')

#### 8.2.2 The Problem of the Shared State
As with any programming paradigm, OOP is a way of restricting our freedom (if you want absolute freedom, use Assembly; no language renders something possible that wasn't available before). This restriction tries to solve the problem of the shared state: multiple programes can modify the same data at once, enabling all sorts of buggs. Also, this makes it harder to modify the code: if, say, the type of a variable changes, you have to remember to change every single function that depended on the data type of that variable.

One possible solution to this problem is the one given by functional programming: “there will be no state”. Another possible solution is the one provided by OOP: “I'll glue together everything that's coupled, so I remember to revise all of them if one changes”.

Let's illustrate this. Say I want procedural code to do the same as the code above. It would be enough to just do the following.

In [ ]:
# Shared global state
balance = 0

def deposit(amount):
    global balance
    balance += amount

def withdraw(amount):
    global balance
    balance -= amount

# Anyone can access and modify balance
deposit(100)
withdraw(40)
print(balance)

This is obviouly gonna be a problem when we add more users, since we have to distinguish between accounts. Also, changes in the logic (e.g., adding different types of accounts) will make complexity increase very rapidely, i.e., this doesn't scale.

In [ ]:
# Shared state: one dictionary holding all account balances
accounts = {
    "alice": 0,
    "bob": 0,
}

def deposit(name, amount):
    accounts[name] += amount

def withdraw(name, amount):
    accounts[name] -= amount  # ❗️No check for overdraft

def get_balance(name):
    return accounts[name]

# Use case
deposit("alice", 100)
withdraw("bob", 20)  # ❗️Invalid: bob has no money
print(f"Alice: {get_balance('alice')}")
print(f"Bob: {get_balance('bob')}")


Say we want to correct for this error and also we're asked to add a history of movements. Now things get more complicated.

In [ ]:
# Problem: tracking savings and checking accounts for many users

# Each account is a tuple:
# (balance: float, type: str, overdraft_allowed: bool, history: list of str)
accounts = {
    "alice_checking": (500.0, "checking", True, []),
    "bob_savings":    (300.0, "savings", False, [])
}

def deposit(account_id, amount):
    bal, acc_type, overdraft, hist = accounts[account_id]
    bal += amount
    hist.append(f"Deposited {amount}")
    accounts[account_id] = (bal, acc_type, overdraft, hist)

def withdraw(account_id, amount):
    bal, acc_type, overdraft, hist = accounts[account_id]
    if not overdraft and bal < amount:
        print("❌ Insufficient funds")
        return
    bal -= amount
    hist.append(f"Withdrew {amount}")
    accounts[account_id] = (bal, acc_type, overdraft, hist)

def print_statement(account_id):
    _, _, _, hist = accounts[account_id]
    print(f"Statement for {account_id}:")
    for h in hist:
        print(" ", h)

# Run a scenario
deposit("alice_checking", 100)
withdraw("alice_checking", 700)  # Allowed due to overdraft
withdraw("bob_savings", 400)     # Should fail
print_statement("alice_checking")


Problems with this:
- Functions are complicated and hard to read (Zen of Python: Readability counts).
- Functions have access to all acounts when modifying the balance of a particular account (prone to error).
- If we want to extend the accounts to include another variable, we'd have to change **all the functions**, and be careful to do it consistently, without forgetting any of them!
- Also, in Python extension of current account tuples is not possible (tupples do not support an append method). However, they were the appropriate data structure (lists would have been prone to error), unless we were willing to nest dictionaries.

Solution: OOP refactor!

In [ ]:
class BankAccount:
    def __init__(self, owner: str, balance: float = 0.0, overdraft_allowed: bool = False, overdraft_limit: float = 0.0):
        self.owner = owner
        self.balance = balance
        self.overdraft_allowed = overdraft_allowed
        self.overdraft_limit = overdraft_limit if overdraft_allowed else 0.0
        self.history = []

    def deposit(self, amount: float):
        self.balance += amount
        self.history.append(f"Deposited {amount:.2f}")

    def withdraw(self, amount: float):
        if self.balance - amount < -self.overdraft_limit:
            raise ValueError("Withdrawal denied: insufficient funds or overdraft limit exceeded")
        self.balance -= amount
        self.history.append(f"Withdrew {amount:.2f}")

    def print_statement(self):
        print(f"Statement for {self.owner}:")
        for line in self.history:
            print(" ", line)
        print(f"Final balance: {self.balance:.2f}")


In [ ]:
alice = BankAccount("Alice", balance=500.0, overdraft_allowed=True, overdraft_limit=300.0)
bob   = BankAccount("Bob", balance=300.0, overdraft_allowed=False)

alice.deposit(100)        # Balance: 600
alice.withdraw(850)       # OK: overdraft limit is 300
try:
    bob.withdraw(400)     # Not allowed
except ValueError as e:
    print(f"Bob: {e}")

alice.print_statement()
print()
bob.print_statement()


Notice how each method always knows what `balance` variable to subtract from or add to: it's just the one of their own object. Thus, related behavior and state are tightly held together in this bundle called “an object”. Moreover, extensions are straight forward. This is OOP's way of solving the problem of the shared state.

### 8.3 Pilars of OOP
Let's now list the of pilars of OOP:
- Abstraction: as with any other type of modeling, we implement only relevant details.
- Encapsulation: details are hidden under the hood, we only interact with a simple interface (e.g. `.deposit()`)
- Inheritance
- Polymorphism

#### 8.3.1 Inheritance
Inheritance is the ability to build new classes on top of existing ones. The main benefit of inheritance is code reuse. They reflect a “is a” type of relationship. For example, in our previous case we could have reused code and further personalize for account type.

In [ ]:
class BankAccount:
    def __init__(self, owner: str, balance: float = 0.0):
        self.owner = owner
        self.balance = balance
        self.history = []

    def deposit(self, amount: float):
        self.balance += amount
        self.history.append(f"Deposited {amount:.2f}")

    def withdraw(self, amount: float):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount
        self.history.append(f"Withdrew {amount:.2f}")

    def print_statement(self):
        print(f"Statement for {self.owner}:")
        for entry in self.history:
            print(" ", entry)
        print(f"Final balance: {self.balance:.2f}")

# CheckingAccount allows overdraft
class CheckingAccount(BankAccount):
    def __init__(self, owner: str, balance: float = 0.0, overdraft_limit: float = 500.0):
        super().__init__(owner, balance)
        self.overdraft_limit = overdraft_limit

    def withdraw(self, amount: float):
        if amount > self.balance + self.overdraft_limit:
            raise ValueError("Overdraft limit exceeded")
        self.balance -= amount
        self.history.append(f"Withdrew {amount:.2f} (checking)")

# SavingsAccount disallows overdraft
class SavingsAccount(BankAccount):
    def withdraw(self, amount: float):
        if amount > self.balance:
            raise ValueError("Insufficient funds (savings)")
        self.balance -= amount
        self.history.append(f"Withdrew {amount:.2f} (savings)")


Notice that, since we haven't further personalized our `SavingsAccount` class, in this case it's redundant and we could have just used the base class.

In [ ]:
alice = CheckingAccount("Alice", balance=500.0)
bob   = SavingsAccount("Bob", balance=300.0)

alice.deposit(100)           # balance = 600
alice.withdraw(700)          # allowed due to overdraft (limit 500)

try:
    bob.withdraw(400)        # disallowed: overdraft not permitted
except ValueError as e:
    print(f"Bob: {e}")

alice.print_statement()
print()
bob.print_statement()


#### 8.3.2 Polymorphism
This is the ability of calling an object with the same message and get class-dependent behavior. To put it plainly, with similarly looking code, you can get different behavior. Let's see a toy example.

In [ ]:
class Animal:
    def make_sound(self):
        print("Some animal sound")

class Dog(Animal):
    def make_sound(self):
        print("¡Guau!")

class Cat(Animal):
    def make_sound(self):
        print("¡Miau!")

a = Animal()
b = Dog()
c = Cat()

a.make_sound()
b.make_sound()
c.make_sound()

Some animal sound
¡Guau!
¡Miau!


In [3]:
# We could iterate over animals without caring
# about exactly which animal we're dealing with
list_of_animals = [a, b, c]

for animal in list_of_animals:
    animal.make_sound()

Some animal sound
¡Guau!
¡Miau!


### 8.4 Features of Good Design
#### 1. Code Reuse
Code reuse is one of the most common ways to reduce development costs. The intent is pretty obvious: instead of developing something over and over from scratch, why don’t we reuse existing code in new projects? Less code to write, less code to read, less code to maintain.

The idea looks great on paper, but it turns out that making existing code work in a new context usually takes extra effort. Tight coupling between components, dependencies on concrete classes instead of interfaces, hardcoded operations—all of this reduces flexibility of the code and makes it harder to reuse it.

Using design patterns is one way to increase flexibility of software components and make them easier to reuse. However, this sometimes comes at the price of making the components more complicated.

#### 2. Extensibility
Change is the only constant thing in a programmer’s life.

That’s why all seasoned developers try to provide for possible future changes when designing an application’s architecture.


### 8.5 Basic Design Principles
What is good software design? How can you make your architecture flexible, stable and easy to understand?

#### 1. Encapsulate What Varies (Modularity)
Identify the aspects of your application that vary and separate them from what stays the same.

**Example:** Encapsulation on a method level.

In [ ]:
# Non-modular

def get_order_total(order):
    total = 0
    for item in order.line_items:
        total += item.price * item.quantity

    if order.country == "US":
        total += total * 0.07  # US sales tax
    elif order.country == "EU":
        total += total * 0.20  # European VAT

    return total

In [ ]:
# Refactored for modularity

def get_order_total(order):
    total = 0
    for item in order.line_items:
        total += item.price * item.quantity
    total += total * get_tax_rate(order.country)
    return total

def get_tax_rate(country):
    if country == "US":
        return 0.07  # US sales tax
    elif country == "EU":
        return 0.20  # European VAT
    else:
        return 0.0

**Example:** Encapsulation on a class level.

Over time, you might add more and more responsibilities to a method which used to do a simple thing. These added behaviors often come with their own helper fields and methods that eventually blur the primary responsibility of the containing class.

Extracting everything to a new class might make things much more clear and simple.

In [ ]:
# SRP violation - tax logic in order class
class Order:
    def __init__(self, line_items, country, state, city):
        self.line_items = line_items
        self.country = country
        self.state = state
        self.city = city
        # ... 20+ other fields

    def get_order_total(self):
        total = 0
        for item in self.line_items:
            subtotal = item.price * item.quantity
            total += subtotal + (subtotal * self.get_tax_rate(self.country, self.state, item.product))
        return total

    def get_tax_rate(self, country, state, product):
        # tax logic grows here — violating SRP
        if country == "US":
            return self._get_us_tax(state)
        elif country == "EU":
            return self._get_eu_tax(country)
        elif country == "CN":
            return self._get_chinese_tax(product)
        return 0.0

    def _get_us_tax(self, state):
        if state == "CA":
            return 0.075
        elif state == "NY":
            return 0.088
        return 0.07

    def _get_eu_tax(self, country):
        return 0.20

    def _get_chinese_tax(self, product):
        return 0.13 if product.is_imported else 0.09


**Refactoring:** Objects of the `Order` class delegate all tax-related work to a special object that does just that.

In [ ]:
class Order:
    def __init__(self, line_items, country, state, city, tax_calculator):
        self.line_items = line_items
        self.country = country
        self.state = state
        self.city = city
        self.tax_calculator = tax_calculator
        # ... other fields

    def get_order_total(self):
        total = 0
        for item in self.line_items:
            subtotal = item.price * item.quantity
            tax_rate = self.tax_calculator.get_tax_rate(self.country, self.state, item.product)
            total += subtotal + (subtotal * tax_rate)
        return total


class TaxCalculator:
    def get_tax_rate(self, country, state, product):
        if country == "US":
            return self.get_us_tax(state)
        elif country == "EU":
            return self.get_eu_tax(country)
        elif country == "CN":
            return self.get_chinese_tax(product)
        return 0.0

    def get_us_tax(self, state):
        return {
            "CA": 0.075,
            "NY": 0.088
        }.get(state, 0.07)

    def get_eu_tax(self, country):
        return 0.20  # simplification

    def get_chinese_tax(self, product):
        return 0.13 if product.is_imported else 0.09


#### 2. Program to an Interface, not an Implementation
What does a class really need to know about their collaborators? Surely nothing about the internal structure or the concrete implementation.

Show my output class on `tumorsphere_culture`.

#### 3. Favor Composition Over Inheritance
Inheritance is probably the most obvious and easy way of reusing code between classes. You have two classes with the same code. Create a common base class for these two and move the similar code into it.

Unfortunately, inheritance comes with caveats that often become apparent only after your program already has tons of classes and changing anything is pretty hard. Here’s a list of those problems.
- **A subclass can’t reduce the interface of the superclass:** You have to implement all abstract methods of the parent class even if you won’t be using them.
- **When overriding methods you need to make sure that the new behavior is compatible with the base one.**
- **Inheritance breaks encapsulation of the superclass:** Internal details of the parent class become available to the subclass.
- **Subclasses are tightly coupled to superclasses:** Any change in a superclass may break the functionality of subclasses.
- **Trying to reuse code through inheritance can lead to creating parallel inheritance hierarchies:** Inheritance usually takes place in a single dimension. But whenever there are two or more dimensions, you have to create lots of class combinations, bloating the class hierarchy to a ridiculous size.


This is why in `tumorsphere_culture` a `Culture` *has* an `Output` object, instead of having classes with each type of output.

Let's see an example.


In [ ]:
# Base class
class Culture:
    pass

# Dimension 1: boundary conditions
class UnboundedCulture(Culture):
    pass

class PeriodicBoundaryCulture(Culture):
    pass

# Dimension 2: output type
class PopulationsMeasuringUnboundedCulture(UnboundedCulture):
    pass

class PositionsMeasuringUnboundedCulture(UnboundedCulture):
    pass

class PopulationsMeasuringPeriodicBoundaryCulture(PeriodicBoundaryCulture):
    pass

class PositionsMeasuringPeriodicBoundaryCulture(PeriodicBoundaryCulture):
    pass


This is hell. A better option would be to use composition:

In [ ]:
class BoundaryConditions:
    pass

class Output:
    pass

class Culture:
    def __init__(self, boundary_conditions, output):
        self.boundary_conditions = boundary_conditions
        self.output = output

# In principle, Output and BoundaryConditions would then have subclasses,
# but an even better idea is to use abstract classes as interfaces.

### 8.6 SOLID Principles
Robert Martin introduced them in the book *Agile Software Development, Principles, Patterns, and Practices*.


#### **S -** Single Responsibility Principle

> A class should have just one reason to change.

Try to make every class responsible for a single part of the functionality provided by the software, and make that responsibility entirely encapsulated by (you can also say *hidden within*) the class.


#### **O -** Open/Closed Principle

> Classes should be open for extension but closed for modification.

A class is *open if you can extend it*, produce a subclass and do whatever you want with it—add new methods or fields, override base behavior, etc.

At the same time, the class is *closed* (you can also say *complete*) if it’s 100% ready to be used by other classes — its *interface is clearly defined and won’t be changed* in the future.


#### **L -** Liskov Substitution Principle

> When extending a class, remember that you should be able to pass objects of the subclass in place of objects of the parent class without breaking the client code.

Unlike other design principles which are wide open for interpretation, the substitution principle has a set of formal requirements for subclasses, and specifically for their methods. Let’s go over this checklist in detail.
- Parameter types in a method of a subclass should match or be more abstract than parameter types in the method of the superclass.
- The return type in a method of a subclass should match or be a subtype of the return type in the method of the superclass.
- A method in a subclass shouldn’t throw types of exceptions which the base method isn’t expected to throw.
- A subclass shouldn’t strengthen pre-conditions.
- A subclass shouldn’t weaken post-conditions.
- Invariants of a superclass must be preserved. (Invariants are conditions in which an object makes sense.)
- A subclass shouldn’t change values of private fields of the superclass.

Note: now this principle is usually built into languages, but I think SmallTalk didn't abide by it for instance.


#### **I -** Interface Segregation Principle

> Clients shouldn’t be forced to depend on methods they do not use.

Try to make your interfaces narrow enough that client classes don’t have to implement behaviors they don’t need.

According to the interface segregation principle, you should break down “fat” interfaces into more granular and specific ones. Clients should implement only those methods that they really need. Otherwise, a change to a “fat” interface would break even clients that don’t use the changed methods.


#### **D -** Dependency Inversion Principle

> High-level classes shouldn’t depend on low-level classes. Both should depend on abstractions. Abstractions shouldn’t depend on details. Details should depend on abstractions.

Basically, insert dependencies into a level as high as possible.




### 8.7 Design Patterns
#### 8.7.1 What's a Design Pattern?
Design patterns are typical solutions to commonly occurring problems in software design. They are like pre-made blueprints that you can customize to solve a recurring design problem in your code.

You can’t just find a pattern and copy it into your program, the way you can with off-the-shelf functions or libraries. The pattern is not a specific piece of code, but a general concept for solving a particular problem. You can follow the pattern details and implement a solution that suits the realities of your own program.


#### 8.7.2 Classification of patterns
From low to high level:
- Idioms: The most basic and low-level patterns, they usually apply only to a single programming language.
- Sweet middle ground (we're gonna focus on these)
- Architectural Patterns: The most universal and high-level patterns. Developers can implement these patterns in virtually any language. Unlike other patterns, they can be used to design the architecture of an entire application.

By intent:
- **Creational patterns** provide object creation mechanisms that increase flexibility and reuse of existing code.
- **Structural patterns** explain how to assemble objects and classes into larger structures, while keeping these structures flexible and efficient.
- **Behavioral patterns** take care of effective communication and the assignment of responsibilities between objects.


#### 8.7.3 Why Should We Learn Patterns?

1. Design patterns are a toolkit of **tried and tested solutions** to common problems in software design. Their *strong points* as well as their *disadvantages are well known*. Even if you never
encounter these problems, knowing patterns is still useful because it teaches you how to solve all sorts of problems using principles of object-oriented design.

2. Design patterns define a **common language** that you and your teammates can use to communicate more efficiently. You can say, “Oh, just use a Singleton for that,” and everyone will
understand the idea behind your suggestion.


#### 8.7.4 Selected Patterns

##### Strategy (Patrón de Comportamiento)
El patrón Strategy sugiere que tomes esa clase que hace algo específico de muchas formas diferentes y extraigas todos esos algoritmos para colocarlos en clases separadas llamadas estrategias, intercambiables en run time.

La clase original, llamada contexto, almacena una referencia a una de las estrategias.

El contexto delega el trabajo a un objeto de estrategia vinculado en lugar de ejecutarlo por su cuenta.

La clase contexto no es responsable de seleccionar un algoritmo adecuado para la tarea.

Veamos un ejemplo de juguete.

In [ ]:
class Moverse:  # context
    def __init__(self, animal):
        self.strategy = animal
    def execute(self):
        print("!!! hago cosas importantes")
        self.strategy.moverse()
        print("!!! hago cosas importantes después")

# strategies =========================================
import abc
        
class StrategyABC(abc.ABC):
    @abc.abstractmethod
    def moverse(self): ...
    
class Perro(StrategyABC):
    def moverse(self):
        print("Camino en cuatro patas")
        
class Pez(StrategyABC):
    def moverse(self):
        print("Nadando")
        
        
ctx = Moverse(Pez())
ctx.execute()
# Notar que en cualquier momento podría cambiar de estrategia
# ctx.strategy = Perro()

!!! hago cosas importantes
Nadando
!!! hago cosas importantes después


In [7]:
ctx2 = Moverse(Perro())
ctx2.execute()

!!! hago cosas importantes
Camino en cuatro patas
!!! hago cosas importantes después


En un lenguaje moderno como Python, muchos de los patrones originales del libro de the gang of four son triviales de implementar. En particular, el patrón strategy tiene una implementación directa, ya que las funciones son objetos.

In [10]:
def moverse(strategy):
    print("!!! hago cosas importantes")
    strategy()
    print("!!! hago cosas importantes después")

# strategies =========================================
   
def moverse_como_perro():
    print("Camino en cuatro patas")
        
def moverse_como_pez():
    print("Nadando")
        
moverse(moverse_como_pez)

!!! hago cosas importantes
Nadando
!!! hago cosas importantes después


In [11]:
moverse(moverse_como_perro)

!!! hago cosas importantes
Camino en cuatro patas
!!! hago cosas importantes después


El ejemplo más minimalista que se me ocurre es algo del siguiente estilo

In [18]:
def add(x, y): return x + y
def mul(x, y): return x * y

def apply_strategy(strategy, a, b):
    return strategy(a, b)

print(apply_strategy(add, 2, 3))  # 5
print(apply_strategy(mul, 2, 3))  # 6


5
6


Veamos un ejemplo un poco menos de juguete, o al menos un poco más relevante. (Uso clases para poder imponer una interfaz.)

In [17]:
# strategy.py
from abc import ABC, abstractmethod
import numpy as np

class Integrator(ABC):
    @abstractmethod
    def integrate(self, f, y0, t0, dt): ...

class EulerIntegrator(Integrator):
    def integrate(self, f, y0, t0, dt):
        return y0 + dt * f(y0, t0)

class RK4Integrator(Integrator):
    def integrate(self, f, y0, t0, dt):
        k1 = f(y0, t0)
        k2 = f(y0 + 0.5*dt*k1, t0 + 0.5*dt)
        k3 = f(y0 + 0.5*dt*k2, t0 + 0.5*dt)
        k4 = f(y0 + dt*k3, t0 + dt)
        return y0 + dt*(k1 + 2*k2 + 2*k3 + k4)/6

class ODESimulation:
    def __init__(self, f, integrator: Integrator):
        self.f = f
        self.integrator = integrator

    def step(self, y, t, dt):
        return self.integrator.integrate(self.f, y, t, dt)

# usage
if __name__ == "__main__":
    def simple_decay(y, t): return -0.5 * y
    sim_euler = ODESimulation(simple_decay, EulerIntegrator())
    sim_rk4   = ODESimulation(simple_decay, RK4Integrator())

    y0, t0, dt = 1.0, 0.0, 0.1
    print("Euler →", sim_euler.step(y0, t0, dt))
    print("RK4   →", sim_rk4.step(y0, t0, dt))


Euler → 0.95
RK4   → 0.9512294270833334


Algo a destacar de este patrón es que es uno de los tantos abusados por el stack científico.

**Ejemplo:**  scipy

In [12]:
from scipy.spatial import distance

In [13]:
distance.cdist([[1,2,3]], [[10, 20, 30]])

array([[33.67491648]])

In [14]:
distance.cdist([[1,2,3]], [[10, 20, 30]], metric="hamming")


array([[1.]])

In [15]:
distance.cdist([[1,2,3]], [[10, 20, 30]], metric=distance.cityblock)

array([[54.]])

In [16]:
def mi_distancia(u, v, w=None):
    return 42

distance.cdist([[1,2,3]], [[10, 20, 30]], metric=mi_distancia)

array([[42.]])

**Pro's:**
- Podés intercambiar los algoritmos usados dentro de un objeto durante el tiempo de ejecución.
- Podés aislar los detalles de implementación de un algoritmo, del código que lo utiliza.
- Podemos sustituir herencia por composición.
- Cumple principio Open/Close: podemos introducir nuevas estrategias sin tener que cambiar el contexto.

**Con's:**
- Si sólo tenés un par de algoritmos que raramente cambian, no hay razón real para complicar el programa con las nuevas clases o interfaces que vengan con el patrón.
- Los clientes deben conocer las diferencias entre estrategias para poder seleccionar la adecuada.
- Muchos lenguajes de programación modernos tienen un soporte de tipo funcional que te permite implementar distintas versiones de un algoritmo dentro de un grupo de funciones anónimas. Esto permite utilizar dichas funciones exactamente como hubiéramos utilizado los objetos estrategia, pero sin las clases e interfaces adicionales.

---

##### Factory (Patrón Creacional)
El patrón Factory Method sugiere que, en lugar de llamar al operador new para construir objetos directamente, se invoque a un método fábrica especial. Los objetos devueltos por el método fábrica a menudo se denominan productos.

Todos los productos deben tener la misma interfaz.

In [ ]:
# Acá me faltó la interfaz “animal”

class Dog:
    def speak(self): return "woof"

class Cat:
    def speak(self): return "meow"

def animal_factory(kind):
    return {"dog": Dog, "cat": Cat}[kind]()

animal = animal_factory("dog")
print(animal.speak())  # woof


Mostrar `create_output_demux` en `tumorsphere_culture` como ejemplo de uso real como patrón creacional para los objetos output.

**Pro's:**
- Evitás un acomplamiento fuerte entre el creador y los productos concretos.
- Obedece el SRP
- Obedece Open/Close Principle: podés incorporar nuevos tipos de productos sin comprometer el código existente.

**Con's:**
- Puede ser que el código se complique por la introducción de nuevas clases.

---

##### The Fluent‐Interface (“Method‐Chaining”) Pattern
This might actually be considered an **API design idiom**, but I decided to include it cause it's also widely used in scientific computing, in particular in data science, but also whenever your building pipelines.

Intent: have each operation return self (or a new instance) so you can express pipelines in a single, readable chain—just like with Pandas or Matplotlib.

Why it matters: your data‐processing or simulation pipelines read like a script, not a nested forest of temporaries.

In [ ]:
# Data Processing Pipeline Example
import numpy as np

class DataPipeline:
    def __init__(self, data: np.ndarray):
        self.data = data.copy()

    def filter_positive(self):
        self.data = self.data[self.data > 0]
        return self

    def normalize(self):
        mn, mx = self.data.min(), self.data.max()
        self.data = (self.data - mn) / (mx - mn)
        return self

    def aggregate(self, func):
        self._result = func(self.data)
        return self

    def result(self):
        return getattr(self, "_result", self.data)

if __name__ == "__main__":
    raw = np.array([3, -1, 0, 5, 2])
    res = (DataPipeline(raw)
           .filter_positive()
           .normalize()
           .aggregate(np.mean)
           .result())
    print("Final result:", res)


Notar que esto es lo que hace `pandas` para que podamos encadenar filtrados, agregados, y procesamientos:

In [ ]:
import pandas as pd

df = pd.read_csv("data.csv")
df = df.dropna(subset=["temperature"]).assign(temp_C=lambda d: (d.temperature - 32) * 5/9).query("temp_C > 20")


In [ ]:

df = (
    pd.read_csv("measurements.csv")
      .query("temperature > 300")
      .groupby("material")["stress"]
      .agg(["mean", "std"])
      .reset_index()
)


In [ ]:
# Image Processing Pipeline Example
import numpy as np
from skimage.color import rgb2gray
from skimage.filters import gaussian
from skimage.feature import canny

class ImageProcessor:
    def __init__(self, img: np.ndarray):
        self.img = img.copy()

    def to_grayscale(self):
        self.img = rgb2gray(self.img)
        return self

    def blur(self, sigma: float):
        self.img = gaussian(self.img, sigma=sigma)
        return self

    def detect_edges(self):
        self.img = canny(self.img)
        return self

    def get(self) -> np.ndarray:
        return self.img

# Usage:
# raw_img is a (H, W, 3) RGB array
processed = (
    ImageProcessor(raw_img)
      .to_grayscale()     # convert to gray
      .blur(sigma=1.5)    # smooth noise
      .detect_edges()     # extract contours
      .get()              # retrieve result
)


#### The Mixin Pattern (Patrón Creacional)
(Sólo por mostrar un patrón un poquito más avanzado.)

Supongamos que estamos creando clases para distintos compuestos químicos. Los compuestos tienen unas pocas propiedades distintivas. Sin embargo, el número total de compuestos posibles es el número de combinaciones de estas propiedades. Como este número es grande, queremos evitar de alguna manera el tener que crear una clase por compuesto.

El patrón Mixin nos sugiere que si hay algo como “una descomposición ortogonal” del total de objetos en estas propiedades, podemos crear as needed los compuestos via herencia múltiple de las propiedades.

In [19]:
class PropertyMixin:
    @classmethod
    def get_properties(cls):
        return []

class FlammableMixin(PropertyMixin):
    @classmethod
    def get_properties(cls):
        return ["flammable"]

class ToxicMixin(PropertyMixin):
    @classmethod
    def get_properties(cls):
        return ["toxic"]

class CorrosiveMixin(PropertyMixin):
    @classmethod
    def get_properties(cls):
        return ["corrosive"]

class Chemical:
    def __init__(self, name):
        self.name = name

    def describe(self):
        props = set()
        for cls in self.__class__.mro():
            if hasattr(cls, "get_properties"):
                props.update(cls.get_properties())
        return f"{self.name} is: {', '.join(props) or 'inert'}."


In [20]:
class Ethanol(FlammableMixin, Chemical): pass
class HydrochloricAcid(CorrosiveMixin, ToxicMixin, Chemical): pass
class Water(Chemical): pass

for chem in [Ethanol("Ethanol"), HydrochloricAcid("HCl"), Water("Water")]:
    print(chem.describe())


Ethanol is: flammable.
HCl is: toxic, corrosive.
Water is: inert.


Note: The **Method Resolution Order** is a list of all classes that Python will search in order when looking for methods and attributes on an object. It includes the class itself, its parents, their parents, and so on, up to `object`.

In [21]:
Ethanol.__mro__

(__main__.Ethanol,
 __main__.FlammableMixin,
 __main__.PropertyMixin,
 __main__.Chemical,
 object)

References:
- (Book) Dive into Design Patterns, by Alexander Shvets.
- (Course) Software Design for Scientific Computing, by Juan Cabral.



Something I wish I had included: Quality for Machine Learning and Patterns for Data Science and Interactive Computing.